# Busqueda del Centro de Masa de la Densidad de Fuentes de Corriente

Hola Banda

In [79]:
muacamuaca=open("DatosActividadEpilepticaSelectos.bin", "r")

IOStream(<file DatosActividadEpilepticaSelectos.bin>)

In [80]:
formaarray=(64,64,9101)

(64,64,9101)

In [81]:
Datos=read(muacamuaca, Float64, formaarray);

In [82]:
tmax=formaarray[3]


9101

No todos los datos son interesentas. La actividad se concentra en los siguientes cuadros.

In [83]:
#No todo es interesante. 
Datos=Datos[:,:,3500:8500]
tmax=size(Datos)[3]

5001

In [84]:
#Ejemplo a comparar
writedlm("ExampleDatosLargos-58-50.dat", reshape(Datos[58,50,:],tmax))

In [85]:
function umbral(x)
    result=((abs(x)>500) ? 0 : x)
end;

function aplastar(x)
    result=((abs(x)<60) ? 0 : x)
end;

function UnNormGauss(x,sigma)
    return exp(-x*x/(2*sigma))
end

UnNormGauss (generic function with 1 method)

In [86]:
close(muacamuaca)

In [87]:
map!(umbral, Datos); #map! quiere decir map in situ
DesviacionPorElectrodo=zeros(64,64)
for j=1:64, k=1:64
    DesviacionPorElectrodo[j,k]=std(reshape(Datos[j,k,:],tmax));
end

In [88]:
function GaussSuavizar(Datos,Sigma)  
    #sigma esta medido en pixeles, es la desviacion estandar de nuestro kernel.
    #El medioancho de nuestra ventana seran 3 sigmas.
    #Bien, veamos como chingaos le ponemos "padding" a las convoluciones.
    medioancho=ceil(Sigma*3)
    colchon=ones(medioancho)
    result=zeros(Datos)
    datoscolchon=vcat(colchon*Datos[1], Datos, colchon*Datos[end])
    kernel=map(x->UnNormGauss(x,Sigma), [-medioancho:medioancho])
    kernel=kernel/(sum(kernel))
    #La convolucion asi normalizada preserva el valor RELATIVO entre los puntos de la funcion.
    #pero queremos ponerlo mas parecido a los voltajes que medimos, para preservar el rango de valores
    #experimentales y su criterio de potenciales de accion / ruido
    for t=medioancho+1:length(Datos)-medioancho
        result[t-medioancho]=sum(datoscolchon[t-medioancho:t+medioancho].*kernel)
    end
    a=maximum(abs(Datos))
    b=maximum(abs(result))
    result=result*a/b
    return result
end
    

GaussSuavizar (generic function with 1 method)

In [89]:
function SigmaVentanas(Datos, ancho, paso)
    anchomedio=floor(ancho/2)
    tantos=convert(Int, floor((length(Datos)-ancho)/paso))
    result=zeros(tantos)
    for t=1:tantos
        result[t]=std(Datos[(t-1)*paso+1:(t-1)*paso+ancho])
    end
    return result  
end

SigmaVentanas (generic function with 1 method)

In [90]:
function OtsuMethod(Data)
    #Pa que esto funcione hay que mandar los datos ya sin NaNs u otras mugres   
    binsdefault=2*ceil(Int,sqrt(length(Data)))
    (rango, cuentas)=hist(Data,binsdefault)
    tantos=length(rango)
    valores
    omega1=0
    omega2=0
    mu1=0
    mu2=0
    sigmab=0
    sigmabtemp=0
    tbest=0
    varlim=0
    for t=1:tantos-1
        omega1=sum(cuentas[1:t])
        omega2=sum(cuentas[t+1:tantos-1])
        mu1=sum(cuentas[1:t].*rango[1:t])/omega1
        mu2=sum(cuentas[t+1:tantos-1].*rango[t+1:tantos-1])/omega2        
        sigmabtemp=omega1*omega2*((mu1-mu2)^2)
        if sigmabtemp>sigmab
            sigmab=sigmabtemp
            tbest=t
            varlim=rango[t]
        end
    end
    return (sigmab,tbest,varlim)
end

function OtsuUmbralizar(DatosMatriz)
    # aplanar datos
    DataFlatten=reshape(DatosMatriz, size(DatosMatriz)[1]*size(DatosMatriz)[2])
    mascara=zeros(DatosMatriz)
    umbral=OtsuMethod(DataFlatten)[3]
    mascara=map(x->(x>umbral)?1:0, DatosMatriz)
    return mascara
end


OtsuUmbralizar (generic function with 1 method)

In [91]:
ancho=350
paso=35
cuantasventanas=convert(Int, floor((tmax-ancho)/paso))
DesviacionPorVentana=zeros((64,64,cuantasventanas))
for j=1:64, k=1:64
    chorizo=reshape(Datos[j,k,:],tmax)
    DesviacionPorVentana[j,k,:]=SigmaVentanas(chorizo, ancho, paso)
end

In [92]:
SumaVentanasOtsuUmbral=zeros(64,64)
for t=1:cuantasventanas
    SumaVentanasOtsuUmbral+=OtsuUmbralizar(DesviacionPorVentana[:,:,t])
end

In [93]:
writedlm("MascaraOtsuRica01.dat", SumaVentanasOtsuUmbral)

In [94]:
MascaraOtsu=zeros(64,64);
MascaraOtsu=map(x->(x>120)?1:0, SumaVentanasOtsuUmbral);

In [95]:
function ParchaMascara01(Datos)
    result=zeros(Datos)
    temp=copy(Datos)
    (alto,ancho)=size(Datos)
    #Primero, hacemos el padding de los datos para que no se suavice demasiado
    
    colchonvertical=zeros(1,ancho)
    colchonhorizontal=zeros(alto+2)
    
    temp=vcat(colchonvertical, temp, colchonvertical)
    temp=hcat(colchonhorizontal, temp, colchonhorizontal)
    
    
    for j=2:alto+1, k=2:ancho+1
        aux=temp[j-1:j+1,k-1:k+1]
        test=sum(aux)
        if(temp[j,k]<2.0 && test<2.1)
            result[j-1,k-1]=0
        #    println(j,k)
        elseif(temp[j,k]<1.1 && test>2.0)
            result[j-1,k-1]=1
        else
            result[j-1,k-1]=temp[j,k]
        end
    end   

    return result
    
end


ParchaMascara01 (generic function with 1 method)

In [96]:
MascaraOtsu=ParchaMascara01(MascaraOtsu);

In [97]:
DatosLimpios=zeros(64,64,tmax)
for t=1:tmax
    DatosLimpios[:,:,t]=Datos[:,:,t].*MascaraOtsu
end

In [98]:
MascaraOtsu[58,50]

1

In [99]:
writedlm("MascaraOtsu01.dat", MascaraOtsu)

In [100]:
map!(aplastar, DatosLimpios);

In [101]:
Chumbaganga=zeros(DatosLimpios);
for j=1:64, k=1:64
    if(MascaraOtsu[j,k]==1)
        Chumbaganga[j,k,:]=GaussSuavizar(reshape(DatosLimpios[j,k,:],tmax),7)
    end    
end

In [102]:
writedlm("ExampleDatosLargosUmbrales-58-50.dat", reshape(DatosLimpios[58,50,:],tmax))
writedlm("ExampleDatosLargosSuaves-58-50.dat", reshape(Chumbaganga[58,50,:],tmax))

In [26]:
GaussianKernel=readdlm("GaussianMatrix.dat");
function GaussianSmooth(Datos)
    tamanodatos=size(Datos)
    result=zeros(Datos)
    temp=copy(Datos)
    #Primero, hacemos el padding de los datos para que no se suavice demasiado
    for j=1:3
        temp=vcat(temp[1,:], temp, temp[end,:])
    end
    for j=1:3
        temp=hcat(temp[:,1], temp, temp[:,1])
    end
    
    for j=4:tamanodatos[1]+3, k=4:tamanodatos[2]+3
        #los indices van primero, "renglones", luego "columnas", etc
        aux=temp[j-3:j+3,k-3:k+3]
        result[j-3,k-3]=sum(GaussianKernel.*aux)
    end
    #piensa como normalizar
    #result=result*maximum(abs(Datos))/maximum(abs(result))
    return result
end

GaussianSmooth (generic function with 1 method)

In [27]:
Sumbaganga=zeros(formaarray)
for t=1:tmax
    Sumbaganga[:,:,t]=GaussianSmooth(Chumbaganga[:,:,t])
end

In [28]:
extrema(Sumbaganga)

(-218.6969370034172,224.25070894676688)

In [29]:
Datos=0


0

In [30]:
gc()

In [29]:
# a escribir, pa variar
for t=1:tmax
#    aux=map(x-> isnan(x)?0:x, Chumbaganga[:,:,t])
    zacatlan="./GarbageCollector/GaussianFilteredSpaceTime-$t.dat"
    writedlm(zacatlan, Sumbaganga[:,:,t])
end


In [31]:
LaplacianKernel=zeros(3,3);
#checa que Julia va ordenando los datos renglon por renglon, no columna por columna
LaplacianKernel=[[0.5 1. 0.5], [1. -6. 1], [0.5 1. 0.5]];

function DiscreteLaplacian(Datos)
    result=zeros(Datos)
    temp=copy(Datos)
    #Primero, hacemos el padding de los datos para que no se suavice demasiado
    temp=vcat(temp[1,:], temp, temp[end,:])
    temp=hcat(temp[:,1], temp, temp[:,end])
    largo,ancho=size(Datos)
    aux=Array(Float64,(3,3))
    result=zeros(Datos)
    for j=2:largo, k=2:ancho
        #los indices van primero, "renglones", luego "columnas", etc
        aux=temp[j-1:j+1,k-1:k+1]
        result[j-1,k-1]=sum(LaplacianKernel.*aux)
    end
    return result
end

DiscreteLaplacian (generic function with 1 method)

In [32]:
Laplaganga=zeros(formaarray)
for t=1:tmax
    Laplaganga[:,:,t]=DiscreteLaplacian(Sumbaganga[:,:,t])
end

In [34]:
for t=1:tmax
#    aux=map(x-> isnan(x)?0:x, Chumbaganga[:,:,t])
    zacatlan="GarbageCollector/LaplacianFilteredSpaceTime-$t.dat"
    writedlm(zacatlan, Laplaganga[:,:,t])
end


In [35]:
ActividadNegativa=zeros(Laplaganga);
ActividadPositiva=zeros(Laplaganga);
SpikeCountPositivo=zeros(64,64);
SpikeCountNegativo=zeros(64,64);
SpikeCountTotal=zeros(64,64;)
for j=1:64
    for k=1:64
        for t=1:tmax          
            if(Laplaganga[j,k,t]<-0.0)      
                ActividadNegativa[j,k,t]=Laplaganga[j,k,t]
                SpikeCountNegativo[j,k]+=1
            elseif(Laplaganga[j,k,t]>0.0) 
                ActividadPositiva[j,k,t]=Laplaganga[j,k,t]
                SpikeCountPositivo[j,k]+=1
            end
        end
            
    end
end


In [36]:
writedlm("SpikeCountNegativo.dat", SpikeCountNegativo)
writedlm("SpikeCountPositivo.dat", SpikeCountPositivo)

In [37]:
Salida=open("DatosCMNegativo01.dat", "w");
(xanterior, yanterior)=(32., 32.)
for t=1:tmax
    xmasa=0.0
    ymasa=0.0
    for j=1:64
        xmasa+=j*sum(ActividadNegativa[j,:,t])
        ymasa+=j*sum(ActividadNegativa[:,j,t])
    end
    if(abs(masa)>100)
        xmasa=xmasa/masa
        ymasa=ymasa/masa
        write(Salida,join((xmasa,ymasa,masa), "\t"), "\n" )
        xanterior=xmasa
        yanterior=ymasa
    else
        write(Salida,join((xanterior,yanterior,0.0), "\t"), "\n" )
    end
end
close(Salida);


In [38]:
Salida=open("DatosCMPositivo01.dat", "w")
(xanterior,yanterior)=(32.0, 32.0)
for t=1:tmax
    masa=sum(ActividadPositiva[:,:,t]);
    xmasa=0.0
    ymasa=0.0
    for j=1:64
        xmasa+=j*sum(ActividadPositiva[j,:,t])
        ymasa+=j*sum(ActividadPositiva[:,j,t])
    end
    if(abs(masa)>100)
        xmasa=xmasa/masa
        ymasa=ymasa/masa
        write(Salida,join((xmasa,ymasa,masa), "\t"), "\n" )
        xanterior=xmasa
        yanteriro=ymasa
    else 
        write(Salida,join((xanterior,yanterior,0.0), "\t"), "\n" )
    end
        
end
close(Salida);